In [ ]:
# 1. Обновляем pip и устанавливаем инструменты сборки (это помогает при компиляции)
!pip install --upgrade pip setuptools wheel cmake

In [ ]:
!pip install Cython

In [ ]:
# 2. Устанавливаем библиотеки по очереди (так надежнее)
!pip install catboost
!pip install pandas numpy scikit-learn

# 3. Устанавливаем implicit с флагом verbose, чтобы видеть процесс (если будет компилироваться долго)
# Обычно после установки cmake дело идет быстрее
!pip install implicit

In [ ]:
import gc
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import implicit
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import torch

# --- MASTER CONFIGURATION (STRICT MODE) ---
class Config:
    RANDOM_STATE = 42
    VAL_SIZE = 0.1

    # Настройки ALS
    ALS_FACTORS = 64
    ALS_REGULARIZATION = 0.05
    ALS_ITERATIONS = 15
    N_FOLDS_ALS = 5           # 5 фолдов - золотой стандарт

    # Генерация кандидатов
    TOP_K_CANDIDATES = 40     # Hard Negatives от ALS
    NUM_RANDOM_NEGS = 0     # Random Negatives (для баланса)

    # Настройки NLP
    USE_BERT = True         # Включено
    BERT_MODEL = 'cointegrated/rubert-tiny2'
    BERT_PCA_COMPONENTS = 32


    USE_TFIDF=True
    TFIDF_MAX_FEATURES=5000
    TFIDF_SVD_COMPONENTS=24



    # Фичи
    USE_META_FEATURES = True
    USE_LOO_STATS = True      # Оставляем Leave-One-Out (безопасно)
    USE_TIME_FEATURES = True

    CAT_COLS = ['author_id', 'publisher', 'language', 'category_id']

CFG = Config()
print("Strict Config loaded.")

# Функция оптимизации памяти (Обязательная)
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object and not str(col_type).startswith('datetime'):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    print(f'Memory usage reduced to {df.memory_usage().sum() / 1024**2:.2f} MB')
    return df

print("Config loaded.")

In [ ]:
print("Loading data...")
# Замените пути на свои
train_df = pd.read_csv('train.csv', parse_dates=['timestamp'])
candidates_df = pd.read_csv('candidates.csv')
users_df = pd.read_csv('users.csv')
books_df = pd.read_csv('books.csv')
book_descriptions = pd.read_csv("book_descriptions.csv")

# Оптимизация
train_df = reduce_mem_usage(train_df)
users_df = reduce_mem_usage(users_df)
books_df = reduce_mem_usage(books_df)

# Глобальные маппинги ID (важно для ALS)
unique_users = train_df['user_id'].unique()
unique_items = books_df['book_id'].unique()

user2idx = {u: i for i, u in enumerate(unique_users)}
item2idx = {i: k for k, i in enumerate(unique_items)}
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {k: i for i, k in item2idx.items()}

num_users = len(unique_users)
num_items = len(unique_items)

print(f"Users: {num_users}, Items: {num_items}")

In [ ]:
def process_text_features(desc_df):
    print("--- Processing Text Features ---")
    desc_df['description'] = desc_df['description'].fillna('')
    final_emb_df = pd.DataFrame({'book_id': desc_df['book_id']})

    # 1. TF-IDF + SVD
    if CFG.USE_TFIDF:
        print("Running TF-IDF + SVD...")
        tfidf = TfidfVectorizer(max_features=CFG.TFIDF_MAX_FEATURES, stop_words='english')
        tfidf_mat = tfidf.fit_transform(desc_df['description'])

        svd = TruncatedSVD(n_components=CFG.TFIDF_SVD_COMPONENTS, random_state=CFG.RANDOM_STATE)
        svd_mat = svd.fit_transform(tfidf_mat)

        cols = [f'tfidf_svd_{i}' for i in range(CFG.TFIDF_SVD_COMPONENTS)]
        temp_df = pd.DataFrame(svd_mat, columns=cols)
        final_emb_df = pd.concat([final_emb_df, temp_df], axis=1)

    # 2. BERT + PCA
    if CFG.USE_BERT:
        print(f"Running BERT ({CFG.BERT_MODEL})...")
        model_bert = SentenceTransformer(CFG.BERT_MODEL)
        # Если есть GPU, используем его
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model_bert.to(device)

        # Получаем эмбеддинги (это может занять время, но rubert-tiny быстрый)
        embeddings = model_bert.encode(
            desc_df['description'].tolist(),
            show_progress_bar=True,
            batch_size=64,
            convert_to_numpy=True
        )

        # Сжимаем PCA
        print("Reducing BERT dimensions with PCA...")
        pca = PCA(n_components=CFG.BERT_PCA_COMPONENTS, random_state=CFG.RANDOM_STATE)
        bert_pca = pca.fit_transform(embeddings)

        cols = [f'bert_pca_{i}' for i in range(CFG.BERT_PCA_COMPONENTS)]
        temp_df = pd.DataFrame(bert_pca, columns=cols)
        final_emb_df = pd.concat([final_emb_df, temp_df], axis=1)

    return reduce_mem_usage(final_emb_df)

# Запускаем обработку текста
if book_descriptions is not None:
    text_features_df = process_text_features(book_descriptions)
else:
    text_features_df = None

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit
from sklearn.model_selection import KFold
import gc
import torch

# Маппинги должны быть уже созданы ранее, но на всякий случай обновим их глобально
unique_users = train_df['user_id'].unique()
unique_items = books_df['book_id'].unique()
user2idx = {u: i for i, u in enumerate(unique_users)}
item2idx = {i: k for k, i in enumerate(unique_items)}
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {k: i for i, k in item2idx.items()}
num_users, num_items = len(unique_users), len(unique_items)

def generate_hybrid_train_data_v8(df, n_hard=40, n_random=10, n_folds=5):
    print(f"--- [v8] Generating Hybrid Train Data (Global Candidates + OOF Scores) ---")

    # === ЭТАП 1: Обучение Глобальной ALS (для выбора кандидатов) ===
    print("1. Training Global ALS (Candidate Selection)...")
    rows = df['user_id'].map(user2idx).values
    cols = df['book_id'].map(item2idx).values
    # Создаем полную матрицу взаимодействий
    sparse_global = sparse.csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(num_users, num_items))

    global_model = implicit.als.AlternatingLeastSquares(
        factors=CFG.ALS_FACTORS,
        regularization=CFG.ALS_REGULARIZATION,
        iterations=CFG.ALS_ITERATIONS,
        random_state=CFG.RANDOM_STATE,
        use_gpu=torch.cuda.is_available()
    )
    global_model.fit(sparse_global, show_progress=True)

    # === ЭТАП 2: Сборка структуры датасета (Строки) ===
    print(f"2. Constructing Dataset Skeleton (Pos + {n_hard} Hard + {n_random} Random)...")

    # А) Позитивы (Target: 2 = Read, 1 = Planned)
    pos_df = df[['user_id', 'book_id', 'has_read']].copy()
    pos_df['target'] = pos_df['has_read'].apply(lambda x: 2.0 if x == 1 else 1.0)
    pos_df.drop(columns=['has_read'], inplace=True)

    # Б) Hard Negatives (Target: 0) - берем топ рекомендаций глобальной модели
    # Это самые "сложные" примеры, где модель ошибается
    all_users_idx = np.arange(num_users)
    ids, _ = global_model.recommend(
        all_users_idx,
        sparse_global[all_users_idx],
        N=n_hard,
        filter_already_liked_items=True
    )

    neg_data = []
    # Разворачиваем рекомендации
    for i, u_idx in enumerate(all_users_idx):
        u_id = idx2user[u_idx]
        for b_idx in ids[i]:
            # Target 0
            neg_data.append([u_id, idx2item[b_idx], 0.0])

    # В) Random Negatives (Target: 0) - для разнообразия и робастности
    rnd_data = []
    if n_random > 0:
        for u_idx in all_users_idx:
            u_id = idx2user[u_idx]
            # Выбираем случайные книги (можно оптимизировать, но цикл надежнее)
            rand_items_idx = np.random.randint(0, num_items, n_random)
            for r_idx in rand_items_idx:
                rnd_data.append([u_id, idx2item[r_idx], 0.0])

    # Сборка
    neg_df = pd.DataFrame(neg_data, columns=['user_id', 'book_id', 'target'])
    rnd_df = pd.DataFrame(rnd_data, columns=['user_id', 'book_id', 'target'])

    full_df = pd.concat([pos_df, neg_df, rnd_df], ignore_index=True)

    # Удаляем дубликаты (Приоритет: Pos > Hard > Random)
    full_df.sort_values(by='target', ascending=False, inplace=True)
    full_df.drop_duplicates(subset=['user_id', 'book_id'], keep='first', inplace=True)

    # Инициализируем колонки для OOF скоров (пока пустые)
    full_df['als_score'] = np.nan
    full_df['als_item_norm'] = np.nan

    # Помечаем реальные позитивы, чтобы знать, что исключать при OOF обучении
    # Создаем set для быстрой проверки: (u_idx, b_idx)
    real_interactions = set(zip(
        pos_df['user_id'].map(user2idx).values,
        pos_df['book_id'].map(item2idx).values
    ))

    print(f"Skeleton created. Rows: {len(full_df)}. Starting OOF Scoring...")

    # === ЭТАП 3: OOF Scoring (Честный расчет скоров) ===
    # Разбиваем ВЕСЬ full_df на фолды. Для каждого фолда обучаем ALS заново,
    # ИСКЛЮЧАЯ из обучения те позитивы, которые попали в валидацию.

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=CFG.RANDOM_STATE)

    # Исходные данные для матрицы (все взаимодействия)
    train_u_indices = df['user_id'].map(user2idx).values
    train_b_indices = df['book_id'].map(item2idx).values

    fold_cnt = 0
    for _, val_idx in kf.split(full_df):
        fold_cnt += 1

        # Строки, которые мы сейчас будем скорить
        current_rows = full_df.iloc[val_idx]

        # Находим среди них те, которые являются РЕАЛЬНЫМИ позитивами
        # Их нужно "спрятать" от ALS
        current_u_idxs = current_rows['user_id'].map(user2idx).fillna(-1).astype(int).values
        current_b_idxs = current_rows['book_id'].map(item2idx).fillna(-1).astype(int).values

        # Собираем пары (u, i), которые надо исключить из обучения
        pairs_to_exclude = set()
        for u, b in zip(current_u_idxs, current_b_idxs):
            if (u, b) in real_interactions:
                pairs_to_exclude.add((u, b))

        # Строим обучающую матрицу для этого фолда
        # Фильтруем исходный train_df
        mask_keep = []
        for u, b in zip(train_u_indices, train_b_indices):
            if (u, b) in pairs_to_exclude:
                mask_keep.append(False)
            else:
                mask_keep.append(True)

        mask_keep = np.array(mask_keep)

        # Создаем разреженную матрицу (без валидационных позитивов)
        sparse_fold = sparse.csr_matrix(
            (np.ones(mask_keep.sum()), (train_u_indices[mask_keep], train_b_indices[mask_keep])),
            shape=(num_users, num_items)
        )

        # Обучаем ALS для фолда
        model_fold = implicit.als.AlternatingLeastSquares(
            factors=CFG.ALS_FACTORS, regularization=CFG.ALS_REGULARIZATION,
            iterations=CFG.ALS_ITERATIONS, random_state=CFG.RANDOM_STATE,
            use_gpu=torch.cuda.is_available()
        )
        model_fold.fit(sparse_fold, show_progress=False)

        # Предсказываем (Dot Product)
        if hasattr(model_fold.user_factors, "to_numpy"):
            u_fact = model_fold.user_factors.to_numpy()
            i_fact = model_fold.item_factors.to_numpy()
        else:
            u_fact = model_fold.user_factors
            i_fact = model_fold.item_factors

        # Берем вектора для текущих юзеров и книг (даже если юзер был удален из трейна, вектор будет, хоть и плохой)
        # Обработка -1 не нужна, так как skeleton построен на известных ID
        vectors_u = u_fact[current_u_idxs]
        vectors_b = i_fact[current_b_idxs]

        scores = (vectors_u * vectors_b).sum(axis=1)
        norms = np.linalg.norm(vectors_b, axis=1)

        # Записываем
        full_df.iloc[val_idx, full_df.columns.get_loc('als_score')] = scores
        full_df.iloc[val_idx, full_df.columns.get_loc('als_item_norm')] = norms

        print(f"Fold {fold_cnt}/{n_folds} scored.")

        # Чистим память
        del model_fold, sparse_fold, vectors_u, vectors_b
        gc.collect()

    full_df = full_df.sample(frac=1, random_state=CFG.RANDOM_STATE).reset_index(drop=True)
    print(f"Hybrid Dataset v8 Ready.\nTarget Counts:\n{full_df['target'].value_counts()}")

    return reduce_mem_usage(full_df)

In [ ]:
# Предварительный расчет ГЛОБАЛЬНЫХ статистик (один раз на весь ноутбук)
# Добавляем "дочитываемость" (read_ratio) - сильная фича из v4
global_book_stats = train_df.groupby('book_id').agg({
    'rating': ['mean', 'count'],
    'has_read': ['mean', 'sum']
})
global_book_stats.columns = ['book_mean', 'book_count', 'book_read_ratio', 'book_read_sum']

global_user_stats = train_df.groupby('user_id').agg({
    'rating': ['mean', 'count'],
    'has_read': ['mean', 'sum']
})
global_user_stats.columns = ['user_mean', 'user_count', 'user_read_ratio', 'user_read_sum']

# Глобальные средние для заполнения пропусков
GLOBAL_MEAN_RATING = train_df['rating'].mean()
GLOBAL_MEAN_READ = train_df['has_read'].mean()

def feature_engineering_v8(df):
    print(f"--- Feature Engineering v8 (Consistent/No-LOO) ---")
    out_df = df.copy()

    # 1. Meta Features
    if CFG.USE_META_FEATURES:
        out_df = out_df.merge(users_df, on='user_id', how='left')
        out_df = out_df.merge(books_df, on='book_id', how='left')
        for c in CFG.CAT_COLS:
            if c in out_df.columns:
                out_df[c] = out_df[c].fillna(-1).astype(int)

    # 2. Text Features (BERT/TF-IDF)
    # Используем уже подготовленный text_features_df
    if 'text_features_df' in globals() and text_features_df is not None:
        out_df = out_df.merge(text_features_df, on='book_id', how='left')
        feat_cols = [c for c in text_features_df.columns if c != 'book_id']
        out_df[feat_cols] = out_df[feat_cols].fillna(0.0)

    # 3. Statistics (Global Merge)
    # Просто мержим. НЕ делаем вычитание (LOO).
    # Это создает небольшой лик на трейне, но обеспечивает совпадение с тестом.
    out_df = out_df.merge(global_book_stats, on='book_id', how='left')
    out_df = out_df.merge(global_user_stats, on='user_id', how='left')

    # Заполнение пропусков
    out_df['book_mean'] = out_df['book_mean'].fillna(GLOBAL_MEAN_RATING)
    out_df['book_count'] = out_df['book_count'].fillna(0)
    out_df['book_read_ratio'] = out_df['book_read_ratio'].fillna(GLOBAL_MEAN_READ)
    out_df['book_read_sum'] = out_df['book_read_sum'].fillna(0)

    out_df['user_mean'] = out_df['user_mean'].fillna(GLOBAL_MEAN_RATING)
    out_df['user_count'] = out_df['user_count'].fillna(0)
    out_df['user_read_ratio'] = out_df['user_read_ratio'].fillna(GLOBAL_MEAN_READ)
    out_df['user_read_sum'] = out_df['user_read_sum'].fillna(0)

    # 4. Time Features
    if CFG.USE_TIME_FEATURES:
        cur_year = 2025
        if 'publication_year' in out_df.columns:
            out_df['publication_year'] = out_df['publication_year'].fillna(cur_year)
            out_df['book_age'] = cur_year - out_df['publication_year']
            # Клипаем выбросы
            out_df['book_age'] = out_df['book_age'].clip(0, 200)

    # Чистка лишнего
    drops = ['title', 'author_name', 'description', 'timestamp', 'has_read']
    out_df.drop([c for c in drops if c in out_df.columns], axis=1, inplace=True)

    return reduce_mem_usage(out_df)

In [ ]:
# 1. Генерируем "честный" обучающий датасет (включая позитивы и негативы)
train_dataset = generate_hybrid_train_data_v8(train_df, n_hard=CFG.TOP_K_CANDIDATES, n_random=CFG.NUM_RANDOM_NEGS, n_folds=CFG.N_FOLDS_ALS)
# 2. Навешиваем фичи (текстовые + статистики с LOO)
# mode='train' включит защиту от лика таргета в статистиках
X_train_strict = feature_engineering_v8(train_dataset)

print("\nReady for CatBoost Training.")
print("Train Shape:", X_train_strict.shape)
print("Columns:", list(X_train_strict.columns))

In [ ]:
print("--- Starting Feature Importance Audit ---")
X_train_full=X_train_strict
# Подготовка данных для аудита
# Сортировка по группам обязательна для YetiRank
X_train_full.sort_values(by='user_id', inplace=True)

audit_cols_drop = ['user_id', 'book_id', 'target']
X_audit = X_train_full.drop(audit_cols_drop, axis=1)
y_audit = X_train_full['target']
group_audit = X_train_full['user_id']

# Находим категориальные колонки, которые реально остались
audit_cat = [c for c in CFG.CAT_COLS if c in X_audit.columns]

# Используем быстрый Ranker
audit_model = CatBoostRanker(
    loss_function='YetiRank',
    iterations=700,        # Мало итераций для скорости
    learning_rate=0.1,
    depth=6,
    task_type='GPU',       # Для аудита можно и CPU, если данных не миллионы
    verbose= 100,
    random_seed=CFG.RANDOM_STATE
)

audit_pool = Pool(
    data=X_audit,
    label=y_audit,
    group_id=group_audit,
    cat_features=audit_cat
)

audit_model.fit(audit_pool)

# Вывод важности
fi = audit_model.get_feature_importance(type='PredictionValuesChange')
fi_df = pd.DataFrame({'feature': X_audit.columns, 'importance': fi}).sort_values('importance', ascending=False)

print("Top 50 Features:")
print(fi_df.head(50))

In [ ]:
# 1. Генерация данных
train_dataset_v8 = generate_hybrid_train_data_v8(train_df, n_hard=CFG.TOP_K_CANDIDATES, n_random=CFG.NUM_RANDOM_NEGS, n_folds=CFG.N_FOLDS_ALS)

# 2. Генерация фичей
X_train_v8 = feature_engineering_v8(train_dataset_v8)

print("\n--- Training CatBoost Ranker (v8) ---")
# Сортировка для групповой обработки
X_train_v8.sort_values(by='user_id', inplace=True)

# Сплит
train_users, val_users = train_test_split(
    X_train_v8['user_id'].unique(),
    test_size=CFG.VAL_SIZE,
    random_state=CFG.RANDOM_STATE
)

train_mask = X_train_v8['user_id'].isin(train_users)
val_mask = X_train_v8['user_id'].isin(val_users)

drop_cols = ['user_id', 'book_id', 'target']
feat_cols = [c for c in X_train_v8.columns if c not in drop_cols]
# Актуализируем список категориальных фичей
cat_feats = [c for c in CFG.CAT_COLS if c in feat_cols]

print(f"Features: {len(feat_cols)}")

train_pool = Pool(
    data=X_train_v8.loc[train_mask, feat_cols],
    label=X_train_v8.loc[train_mask, 'target'],
    group_id=X_train_v8.loc[train_mask, 'user_id'],
    cat_features=cat_feats
)

val_pool = Pool(
    data=X_train_v8.loc[val_mask, feat_cols],
    label=X_train_v8.loc[val_mask, 'target'],
    group_id=X_train_v8.loc[val_mask, 'user_id'],
    cat_features=cat_feats
)

# Параметры v8
# YetiRank отлично работает с градациями 0, 1, 2
params = {
    'loss_function': 'YetiRank',
    'iterations': 4000,
    'learning_rate': 0.05,
    'depth': 8,               # Увеличиваем глубину, т.к. данных стало больше (Hard+Random)
    'l2_leaf_reg': 7,
    'random_seed': CFG.RANDOM_STATE,
    'eval_metric': 'NDCG:top=20',
    'early_stopping_rounds': 150,
    'verbose': 100,
    'task_type': 'GPU' if torch.cuda.is_available() else 'CPU'
}

model_v8 = CatBoostRanker(**params)
model_v8.fit(train_pool, eval_set=val_pool)

print(f"Best Score: {model_v8.get_best_score()}")

In [ ]:
print("--- [v8] Final Inference & Submission ---")

# 1. Готовим кандидатов
candidates_exploded = []
candidates_source = candidates_df.copy()

for _, row in candidates_source.iterrows():
    u_id = row['user_id']
    if pd.isna(row['book_id_list']):
        continue
    b_ids = str(row['book_id_list']).split(',')
    for b_id in b_ids:
        if b_id:
            candidates_exploded.append([u_id, int(b_id)])

test_df = pd.DataFrame(candidates_exploded, columns=['user_id', 'book_id'])
print(f"Test pairs count: {len(test_df)}")

# 2. Считаем ALS Score
print("Retraining Global ALS for Test Scoring...")
all_rows = train_df['user_id'].map(user2idx).values
all_cols = train_df['book_id'].map(item2idx).values
sparse_master = sparse.csr_matrix((np.ones(len(all_rows)), (all_rows, all_cols)), shape=(num_users, num_items))

master_model = implicit.als.AlternatingLeastSquares(
    factors=CFG.ALS_FACTORS, regularization=CFG.ALS_REGULARIZATION,
    iterations=CFG.ALS_ITERATIONS, random_state=CFG.RANDOM_STATE,
    use_gpu=torch.cuda.is_available()
)
master_model.fit(sparse_master, show_progress=False)

# Предсказываем
if hasattr(master_model.user_factors, "to_numpy"):
    u_factors = master_model.user_factors.to_numpy()
    i_factors = master_model.item_factors.to_numpy()
else:
    u_factors = master_model.user_factors
    i_factors = master_model.item_factors

test_u_idxs = test_df['user_id'].map(user2idx).fillna(-1).astype(int).values
test_b_idxs = test_df['book_id'].map(item2idx).fillna(-1).astype(int).values

test_scores = np.zeros(len(test_df))
test_norms = np.zeros(len(test_df))
mask = (test_u_idxs >= 0) & (test_b_idxs >= 0)

if mask.sum() > 0:
    test_scores[mask] = (u_factors[test_u_idxs[mask]] * i_factors[test_b_idxs[mask]]).sum(axis=1)
    test_norms[mask] = np.linalg.norm(i_factors[test_b_idxs[mask]], axis=1)

test_df['als_score'] = test_scores
test_df['als_item_norm'] = test_norms

# 3. Фичи
# !!! ИСПРАВЛЕНИЕ ЗДЕСЬ !!!
# Мы перезаписываем test_df, чтобы учесть возможное увеличение строк после merge
test_df = feature_engineering_v8(test_df)
print(f"Shape after Feature Engineering: {test_df.shape}")

# 4. Проверка и выравнивание колонок
expected_cols = model_v8.feature_names_
# Создаем X_test на основе ОБНОВЛЕННОГО test_df
X_test = test_df.copy()

for col in expected_cols:
    if col not in X_test.columns:
        X_test[col] = 0
X_test = X_test[expected_cols]

# 5. Предсказание
print("Predicting with CatBoost...")
final_scores = model_v8.predict(X_test)

# Теперь длины гарантированно совпадают
test_df['score'] = final_scores

# 6. Сборка сабмита
test_df_sorted = test_df.sort_values(['user_id', 'score'], ascending=[True, False])
# Удаляем дубликаты, если они возникли при merge (берем с высшим скором)
test_df_sorted = test_df_sorted.drop_duplicates(subset=['user_id', 'book_id'])

grouped_preds = test_df_sorted.groupby('user_id')['book_id'].apply(list).reset_index()

final_submission = candidates_source[['user_id']].merge(grouped_preds, on='user_id', how='left')

def format_list(x):
    if isinstance(x, list):
        # Топ 20
        return ",".join([str(i) for i in x[:20]])
    return ""

final_submission['book_id_list'] = final_submission['book_id'].apply(format_list)
final_submission = final_submission[['user_id', 'book_id_list']]

# Сохранение
save_name = 'submission_v8_only_hard.csv'
final_submission.to_csv(save_name, index=False)
print(f"Submission saved to {save_name}")
print(final_submission.head())

In [ ]:
# --- ЯЧЕЙКА: СОХРАНЕНИЕ ДАННЫХ НА GOOGLE DRIVE ---
from google.colab import drive
import shutil
import os

# 1. Подключаем Google Диск
print("Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Настройки
# Имя папки на Гугл Диске, куда все сохранится
DRIVE_FOLDER_NAME = 'NTO_RecSys_Data'
DESTINATION_PATH = f'/content/drive/MyDrive/{DRIVE_FOLDER_NAME}'

# Список файлов, которые нужно сохранить (добавь сюда names, если есть другие)
FILES_TO_SAVE = [
    'train.csv',
    'candidates.csv',
    'users.csv',
    'books.csv',
    'book_genres.csv',
    'book_descriptions.csv',
    # 'submission_strict_v6_fixed.csv' # раскомментируй, если хочешь сохранить и результат
]

# 3. Создаем папку на диске, если её нет
if not os.path.exists(DESTINATION_PATH):
    os.makedirs(DESTINATION_PATH)
    print(f"Created folder: {DESTINATION_PATH}")
else:
    print(f"Folder exists: {DESTINATION_PATH}")

# 4. Копируем файлы
print("-" * 30)
for file_name in FILES_TO_SAVE:
    if os.path.exists(file_name):
        try:
            shutil.copy2(file_name, f"{DESTINATION_PATH}/{file_name}")
            print(f"✅ Saved: {file_name}")
        except Exception as e:
            print(f"❌ Error saving {file_name}: {e}")
    else:
        print(f"⚠️ File not found in Colab: {file_name}")

print("-" * 30)
print(f"Backup complete. Files are in 'MyDrive/{DRIVE_FOLDER_NAME}'")

In [ ]:
# --- ЯЧЕЙКА: ЗАГРУЗКА ДАННЫХ С GOOGLE DRIVE ---
from google.colab import drive
import shutil
import os

# 1. Подключаем Google Диск
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

# 2. Настройки
# Та же папка, что и в первой ячейке
DRIVE_FOLDER_NAME = 'NTO_RecSys_Data'
SOURCE_PATH = f'/content/drive/MyDrive/{DRIVE_FOLDER_NAME}'
DESTINATION_PATH = '/content'  # Текущая рабочая директория Colab

# 3. Копирование файлов
print("-" * 30)
if os.path.exists(SOURCE_PATH):
    files = os.listdir(SOURCE_PATH)
    count = 0
    for file_name in files:
        # Копируем только файлы (не папки) и исключаем системные файлы
        full_file_name = os.path.join(SOURCE_PATH, file_name)
        if os.path.isfile(full_file_name) and not file_name.startswith('.'):
            shutil.copy2(full_file_name, DESTINATION_PATH)
            print(f"📥 Loaded: {file_name}")
            count += 1

    if count == 0:
        print("Folder is empty.")
    else:
        print("-" * 30)
        print(f"Successfully loaded {count} files from Drive.")
else:
    print(f"❌ Error: Folder '{SOURCE_PATH}' not found on Google Drive.")
    print("Please run the 'Save to Drive' cell first or create the folder manually.")